In [1]:
import cv2
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import time
from IPython.display import Image, clear_output, display
from datetime import datetime

In [8]:
# Step1: Open the defult camera
cam = cv2.VideoCapture(0)

# Step2: Check if the webcam is opened successfully
if not cam.isOpened():
    print("Can not open camera")
    exit()

# Step3: Read and display video frame by frame
while True:
    # Capture frame by frame
    ret, frame = cam.read()
    # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # If frame is read correctly ret is True
    if ret:
        frame = cv2.flip(frame, 1)
        
        # Clear previous output 
        clear_output(wait=True)
        # Displaying an image to windows
        cv2.imshow('GAY', frame)
        # Wait for button press (1 millisecond delay)
        key = cv2.waitKey(1) & 0xFF

        # Pressing the 'q' button to save the image
        if key == ord('q'):
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            img_name = f"Save_Image/img_{timestamp}.jpg"
            cv2.imwrite(img_name, frame)
            print(f"Saved: {img_name}")

        # Pressing the 'ESC' button (ASCII code is 27) will exit the program
        elif key == 27:
            print("Close program")
            break

        # # Break the loop on 'q' key press
        #if cv2.waitKey(1) == ord('q'):
        #     break

        time.sleep(0.01)
    else:
        print("Can not receive frme. Exit the loop")
        break

# Step4: Rdlease the webcam and close all windows
cam.release()
cv2.destroyAllWindows()

Close program


In [9]:

 
# Step1: Defining the dimensions of checkerboard
CHECKERBOARD = (8, 6)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# Step2: Creating vector to store vectors "objpoints" of 3D points for each checkerboard image
objpoints = []
# Step3: Creating vector to store vectors "imgpoints" of 2D points for each checkerboard image
imgpoints = []
 
 
# Step4: Defining the world coordinates for 3D points
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None
 
# Step5: Using function glob.glob to find and returen a list of all file names ending in .jpg in directly folder name /images
images = glob.glob(r'C:\Users\jinny\Desktop\cv\Save_Image\*.jpg')

# Step6: Loop to get the image path in the list.
for fname in images:

    # Step7: Read image
    img = cv2.imread(fname)
    # Step8: Convert RGB to GRAY 
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Step9: Find the chess board corners using "findChessboardCorners"
    # If desired number of corners are found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(
        img_gray,
        CHECKERBOARD,
        cv2.CALIB_CB_ADAPTIVE_THRESH + 
        cv2.CALIB_CB_FAST_CHECK + 
        cv2.CALIB_CB_NORMALIZE_IMAGE

    )
    """
    If desired number of corner are detected,
    we refine the pixel coordinates and display 
    them on the images of checker board
    """
    if ret == True:
        # Step10: Add objp to objpoints list
        objpoints.append(objp)
        # Step11: Refining pixel coordinates for given 2d points.
        corners2 = cv2.cornerSubPix(img_gray, corners, (11,11),(-1,-1), criteria)

        # Step12: Add corners2 to imgpoints list 
        imgpoints.append(corners2)
 
        # Step13: Draw the corners in image
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
     
    # Step14: Display image
    cv2.imshow("image", img)
    # Step15: Delay 500 milliseconds
    cv2.waitKey(500)

# Setp16: Destroy all windows
cv2.destroyAllWindows()

 
"""
Step15: 
Performing camera calibration (calibrateCamera) by 
passing the value of known 3D points (objpoints)
and corresponding pixel coordinates of the 
detected corners (imgpoints)
"""
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
    objpoints, 
    imgpoints, 
    img_gray.shape[::-1], 
    None, 
    None
)

# Display output from camera calibration
print(f"Re-projection Error: \n{ret}")
print(f"\n fCamera matrix : \n{mtx}")
print(f"\n dist : \n{dist}")
print(f"\n rvecs : \n{rvecs}")
print(f"\n tvecs : \n{tvecs}")

# Step17: Save camera matrix 
# 17.1 Using NumPy Array save in .npz file
np.savez("calibration_data.npz", mtx=mtx, dist=dist, rvecs=rvecs, tvecs=tvecs)
print("Save calibration data .npz successfully.")

# 17.2 Using OpenCv format save at .yaml (YAML , XML) file 
cv_file = cv2.FileStorage("calibration_data.yaml", cv2.FILE_STORAGE_WRITE)
cv_file.write("camera_matrix", mtx)
cv_file.write("dist_coeff", dist)
cv_file.release()
print("Save calibration data .yaml successfully.")



Re-projection Error: 
2.9674757381176344

 fCamera matrix : 
[[519.61885652   0.         291.14158475]
 [  0.         529.94441313 306.83821731]
 [  0.           0.           1.        ]]

 dist : 
[[ 0.1424392  -0.37649718  0.02495703  0.02687225 -1.01484266]]

 rvecs : 
(array([[-0.12295336],
       [-0.01548413],
       [-0.02055056]]), array([[-0.22379788],
       [-0.28549633],
       [-0.01385495]]), array([[ 0.44355129],
       [-0.37475071],
       [-0.06146275]]), array([[-0.24065613],
       [ 0.94728919],
       [-0.03460972]]), array([[-0.07531318],
       [-0.03009287],
       [-0.01983617]]), array([[-0.08988976],
       [-0.05199413],
       [-0.01681684]]), array([[-0.52154137],
       [ 0.00721534],
       [-0.0100568 ]]), array([[ 0.11256527],
       [-0.14170685],
       [-0.01424213]]), array([[-0.00238501],
       [-0.19610949],
       [-0.01544146]]), array([[-0.02820989],
       [ 1.00398607],
       [-0.08003002]]), array([[-0.99877894],
       [ 0.13642859],
  

In [14]:
# Step1: Load calbration parameters 
with np.load('calibration_data.npz') as data:
    xtx = data['mtx']
    dist = data['dist']

# Step2: Read the image and get the height and width of the image
img = cv2.imread('Save_Image/img_20251224_154535.jpg')

# Step3: Get the shape (h:height, w:width) from image
h, w = img.shape[:2]

# Step:4: Refine Camera Matrix for optimal framing using "getOptimalNewCameraMatrix" function
new_mtx, roi = cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1, (w,h))

# Step:5. Undistort the image using "undistort" function
dst = cv2.undistort(img, mtx, dist, None, new_mtx)

# Step:6. Crop to usable area 
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
# Step:7. Display the image in windows using opencv
cv2.imshow("image", img)
cv2.imshow("undistort", dst)
# Step8: Delay by waitKey
cv2.waitKey(0)

-1